In [1]:
author="Jumabek Alikhanov"
date = 'Nov 19,2019'

In [2]:
#imports
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
%load_ext autoreload

In [3]:
# load data
dataroot = 'MachineLearningCVE/'
SEED=2

In [ ]:

from preprocessing import load_data
X,y = load_data(dataroot) # reads csv file and returns np array of X,y -> of shape (N,D) and (N,1)

MachineLearningCVE/*.pcap_ISCX.csv
there are 2830743 flow records with 79 feature dimension
stripped col names
dropped bad columns


In [ ]:
from preprocessing import balance_data, normalize
#X = normalize(X)


In [ ]:
# Random Foresty with 10 DecisionTrees
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics

K=5
results = {}
skf = StratifiedKFold(n_splits=K,random_state=SEED)

for fold_index, (dev_index,test_index) in enumerate(skf.split(X,y)): 
    print('---------------------------------------------')
    print('Fold #{}'.format(fold_index))    
    X_dev = X[dev_index]
    y_dev = y[dev_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
    for train_index, val_index in sss.split(X_dev, y_dev): # runs only once
        X_train = X_dev[train_index]
        y_train = y_dev[train_index]
        X_val = X_dev[val_index]
        y_val = y_dev[val_index]
        break
    
    X_train,y_train = balance_data(X_train,y_train)
    clf = RandomForestClassifier(n_estimators=10, random_state=SEED)
    clf.fit(X_train,y_train)
    
    pred = clf.predict(X_val)
    imbalanced_val_acc = metrics.accuracy_score(y_val,pred)*100        
    val_acc = metrics.balanced_accuracy_score(y_val,pred)*100

    pred = clf.predict(X_test)
    test_acc = metrics.balanced_accuracy_score(y_test,pred)*100
    print('val acc:',imbalanced_val_acc)
    print('balanced val acc: ',val_acc)
    print('balanced test acc: ',test_acc)
    
    results[fold_index]= (imbalanced_val_acc, val_acc, test_acc)
    


In [ ]:
sum_imbalanced_val_acc = 0
sum_val_acc = 0
sum_test_acc = 0
for fold_index,res in results.items():
    (acc,balanced_acc,test_acc) = res
    sum_imbalanced_val_acc+=acc
    sum_val_acc +=balanced_acc
    sum_test_acc+= test_acc
print("{}-fold evaluation:".format(K))
print('imbalanced val acc:',sum_imbalanced_val_acc/K)
print('val acc: ',sum_val_acc/K)
print('test acc: ',sum_test_acc/K)
print('{0:.2f} {1:.2f} {2:.2f}'.format(sum_imbalanced_val_acc/K,sum_val_acc/K,sum_test_acc/K))